In [ ]:
from bins import Bins
from utils import calcAllChi2, calcOneChi2, HistMaker
from ROOT import TFile, TH1
import math

bins = Bins.readFrom("ranges.yml")
histMakerData = HistMaker("apr12_diele_088_090_ag123ag_2500A_accepted_np_2.dat", "_data", bins)
histsData = histMakerData.makeHists()
histMakerMC = HistMaker("medium_isotropic_eff_ag1230ag_np_9deg.dat", "_MC", bins    )

#outfile = TFIle("out.root","RECREATE")
allHistsMC = []

#calcAllChi2(histsMC, histsData)

In [ ]:
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

In [ ]:
TH1.SetDefaultSumw2

N_PARAMS = 3

In [ ]:
#lb = -1
#ub = 1

#bounds = torch.tensor([[lb]*N_PARAMS, [ub]*N_PARAMS], dtype=torch.float)
bounds = torch.tensor([[0, 0,         0],
                       [1, 2*math.pi, 2]], dtype=torch.float)

grid_r   = torch.linspace(boundsü[0][0], bounds[1][0], 101)
grid_phi = torch.linspace(boundsü[0][1], bounds[1][1], 101)
grid_z   = torch.linspace(boundsü[0][2], bounds[1][2], 101)

grid_x1, grid_x2, grid_x3 = torch.meshgrid(grid_r, grid_phi, grid_z, indexing="ij")

xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten(), grid_x3.flatten()]).transpose(-2,-1)

In [ ]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [ ]:
num_queries = 200
num_repeats = 1
num_samples = 1

In [ ]:
def lambdas(xx):
   # return x[0], x[1], x[2]
    r, phi, z = xx[0], xx[1], xx[2]
    x = r*math.cos(phi)
    y = r*math.sin(phi)
    lambda_theta = 0.5 * (2*x + z)
    lambda_phi   = 0.25 * (-2 - 2*x + 3*z)
    lambda_theta_phi = y/math.sqrt(2.)
    return lambda_theta, lambda_phi, lambda_theta_phi

def constraint1(xx):
    print("In the constraint fucntion, xx = ", xx)
    input_shape = xx.shape
    ll = list(xx.shape)
    ll[-1] = 1
    if len(input_shape)>1:
        xx = torch.reshape(xx, input_shape[1:])
        output_shape = tuple(ll)
        result = 1+xx[0,0]+xx[0,1]
    else:
        result = 1+xx[0]+xx[1]
        output_shape = (1,)
        
    print(xx)
    print(input_shape)
    print(result)
    print(output_shape)
    
    return torch.reshape(torch.tensor([result],requires_grad=xx.requires_grad),output_shape)

#def cost(xx):
#    def generator(xx):
#        for x in xx:
#            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)#
#
#            result = 0
#
#            if pow(1-lambda_phi, 2) - pow(lambda_theta-lambda_phi, 2) < 4*pow(lambda_theta_phi, 2):
#                result = 1
#            if 1 + lambda_theta + 2*lambda_phi < 0:
#                result = 1

#            yield torch.tensor([result])
#    return torch.stack([a for a in generator(xx)])
    
def objective(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

            histsMC = histMakerMC.makeHists(lambda_theta, lambda_phi, lambda_theta_phi)
            chi2, ndf = calcOneChi2(histsMC[0][0], histsData[0][0])
            allHistsMC.append(histsMC[0][0])
            if not chi2 or not ndf:
                return torch.tensor([0])
            yield torch.tensor([1.0/(chi2 / ndf)])
           # yield torch.tensor([1./(chi2 / ndf)])
    return torch.stack([a for a in generator(xx)])

In [ ]:
# -2 is the default value when no feasible has been found
default_value = -2
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(num_samples, 3)
    train_utility = objective(train_x)

    for i in tqdm(range(num_queries)):
        print("query", i)

        feasible_incumbents[trial, i] = train_utility.max()
        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        best_f = train_utility.max()
                
        policy = botorch.acquisition.monte_carlo.qExpectedImprovement(
            model=utility_model,
            best_f=train_utility.max(),
        )

        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            #    nonlinear_inequality_constraints=[constraint1],
                inequality_constraints=[(torch.tensor([0,2]),torch.tensor([-2.0,-1.0],dtype=torch.float),-2.0)],
            #    ic_generator=botorch.optim.initializers.gen_batch_initial_conditions
            )

        next_utility = objective(next_x)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])


In [ ]:
torch.save(feasible_incumbents, f"./incumbents.pth")
fout = TFile("out.root", "RECREATE")
fout.cd()
for hist in allHistsMC:
    print ("Writing hist: ", hist.GetName())
    hist.Write()
for j, hists in enumerate(histsData):
    for k, hist in enumerate(hists):
            hist.Write()
            print ("Writing hist: ", j, k, hist)
fout.Close()

In [ ]:
t = torch.tensor( [ [-1.0000, -0.4678,  0.5433]])
print(t)
print(t.shape)

In [ ]:
with torch.no_grad():
    predictive_distribution = utility_likelihood(utility_model(xs))
    predictive_mean = predictive_distribution.mean
    predictive_lower, predictive_upper = predictive_distribution.confidence_region()


In [ ]:
from ipywidgets import interact, Layout, IntSlider
import numpy as np

axis_titles = [r"R of cone coordinates", r"$\Phi$ of cone coordinates", r"Z of cone coordinates"]

def oneplot(ax, tensor, index, cmap, title, transpose=None):
        global xs
        tensor_3d = torch.reshape(tensor, (101,101,101))
        xs_3d = torch.reshape(xs, (101,101,101,3))
        if transpose:
               tensor_3d = tensor_3d.transpose(transpose[0],transpose[1])
               xs_3d     = xs_3d    .transpose(transpose[0],transpose[1])
        extent = [-1,1,-1,1]
        if transpose is None:
                print("kuku")
                x_title=axis_titles[1]
                y_title=axis_titles[2]
                extent=[
                        xs_3d[0][0][0][1],
                        xs_3d[0][-1][0][1],
                        xs_3d[0][0][0][2],
                        xs_3d[0][0][-1][2],
                ]
        elif transpose == (0, 1):
                x_title=axis_titles[0]
                y_title=axis_titles[2]
                extent=[
                        xs_3d[0][0][0][0],
                        xs_3d[-1][0][0][0],
                        xs_3d[0][0][0][2],
                        xs_3d[0][0][-1][2],
                ]
        elif transpose == (0,2):
                x_title=axis_titles[1]
                y_title=axis_titles[0]
                extent=[
                        xs_3d[0][0][0][1],
                        xs_3d[0][-1][0][1],
                        xs_3d[0][0][0][0],
                        xs_3d[-1][0][0][0],
                ]

        print("transpose", transpose)
        print("extent", extent)
        pos = ax.imshow(tensor_3d[index], cmap=cmap, interpolation="nearest", origin="lower", 
                vmin=tensor.min(), vmax=tensor.max(), extent=extent)
        ax.set_aspect((extent[1]-extent[0])/(extent[3]-extent[2]))
        ax.set_title(title)
        ax.set_xlabel(x_title)
        ax.set_ylabel(y_title)
        plt.colorbar(pos)



def f(x):
        cmap = "gist_rainbow"
        transpose = None
        fig, ax = plt.subplots(nrows=2, ncols=2)
        oneplot(ax[0][0], predictive_mean, x, cmap, "value", transpose)
        oneplot(ax[1][1], predictive_lower, x, cmap, "lower confidence bound", transpose)
        oneplot(ax[0][1], predictive_upper, x, cmap, "upper confidence bound", transpose)
        oneplot(ax[1][0], predictive_upper-predictive_lower, x, cmap, "confidence int. width", transpose)

interact(f, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))


In [ ]:
def g(x):
        cmap = "gist_rainbow"
        transpose= (0,1)
        fig, ax = plt.subplots(nrows=2, ncols=2)
        oneplot(ax[0][0], predictive_mean, x, cmap, transpose)
        oneplot(ax[0][1], predictive_lower, x, cmap, transpose)
        oneplot(ax[1][1], predictive_upper, x, cmap, transpose)
        oneplot(ax[1][0], predictive_upper-predictive_lower, x, cmap, transpose)

interact(g, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))

In [ ]:
def h(x):
        cmap = "gist_rainbow"
        transpose= (0,2)
        fig, ax = plt.subplots(nrows=2, ncols=2)
        oneplot(ax[0][0], predictive_mean, x, cmap, transpose)
        oneplot(ax[0][1], predictive_lower, x, cmap, transpose)
        oneplot(ax[1][1], predictive_upper, x, cmap, transpose)
        oneplot(ax[1][0], predictive_upper-predictive_lower, x, cmap, transpose)

interact(h, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))

In [ ]:
t = torch.tensor([[[0.0538, 0.2096, 0.2256]]])
t = torch.reshape(t,(1,3))
(1,2,3)[1:]
xs

In [ ]:
(feasible_incumbents==feasible_incumbents.max()).nonzero()

In [ ]:
best_f / (math.pi/2)

In [ ]:
#train_x[94]
lambdas(train_x[46])
#1./math.sqrt(2)
len(xs)
xs_3d = torch.reshape(xs, (101,101,101,3))
xs_3d
